## Use PINN to solve 1D buegers equation

### 1. Burgers Equation

Dne space dimension case:
$$
\frac{\partial u}{\partial t}+u \frac{\partial u}{\partial x}=\nu \frac{\partial^2 u}{\partial x^2}
$$

Domain: 
- space: [-1,1]
- time: [0,1]

IC/BC condition:

$$
u(x,0) = sin(\pi x)\\
u(-1,t) = u(1, t)=0
$$


- non-linear
- discontinuous solution

First, let see the exact solution solved by numerical method:

In [ ]:
import numpy as np
DATA_PATH = "/workspaces/pytorch-PINN/data/center_sine_burgers.npy"
np_data = np.load(DATA_PATH)

In [ ]:
import matplotlib.pyplot as plt

def plot_burgers(np_data, t_idx, num_x=500):
    plt.plot(np_data[t_idx*num_x:(t_idx+1)*num_x, 1], np_data[t_idx*num_x:(t_idx+1)*num_x, 2], label='Numerical')
    plt.xlabel('x')
    plt.ylabel('u')
    plt.title(f'Burgers Equation Solution at t={np_data[t_idx*num_x, 0]:.2f}')
    plt.legend()
    plt.show()

plot_burgers(np_data, t_idx=0)

## 2. PINN

Physics-informed neural networks (PINNs) are a type of universal function approximators that can embed the knowledge of any physical laws that govern a given data-set in the learning process, and can be described by partial differential equations (PDEs).

![PINN](./pictures/Xtfc_scheme.png)

## 3. PINN with Pytorch

To implement a Physics-Informed Neural Network (PINN) using PyTorch, we can divide the code into several functional sections. This allows for a more organized and structured approach to the implementation process:

1. Data section:
    - Generate data in constrain. (In most cases: initial part, boundary part, interior part) 
2. Constrain:
    - Compute loss.
3. Model:
    - Setup neural network to represent pde solver.
4. Train: 
    - training loop.

### data section

In order to solve the equation, we need to prepare three sets of points, which are sampled from the domain of the equation. This will allow us to accurately represent the solution space and obtain accurate results.

Generating data for a PDE by randomly sampling points within specified ranges for the $x$ and $t$ coordinates. The data dictionary contains three keys: "interior", "boundary", and "initial", each of which corresponds to a different type of data.

In [ ]:
# Toy implementation

x_range = [-1, 1]
t_range = [0, 1]

def get_data():
    # Define the interior, boundary, and initial data for the PDE
    data = {}

    # Interior data is randomly generated within the specified x and t ranges
    data["interior"] = {
        "x": np.random.uniform(*x_range, 100),
        "t": np.random.uniform(*t_range, 100),
    }

    # Boundary data is randomly generated along the x-axis within the specified x range, and at a random time within the specified t range
    data["boundary"] = {
        "x": np.random.choice(x_range, 10),
        "t": np.random.uniform(*t_range, 10),
    }

    # Initial data is randomly generated along the x-axis within the specified x range, and at the initial time (t = t_range[0])
    data["initial"] = {
        "x": np.random.uniform(*x_range, 10),
        "t": np.ones(10) * t_range[0],
    }

    return data

The "interior" data is generated by randomly sampling 100 points within the specified $x$ and $t$ ranges. The "boundary" data is generated by randomly sampling 10 points along the $x$-axis within the specified $x$ range, and at a random time within the specified $t$ range. The "initial" data is generated by randomly sampling 10 points along the $x$-axis within the specified $x$ range, and at the initial time $t = 0$.

Show shape and keys in data:

In [ ]:
import tree

data = get_data()
tree.map_structure(lambda x: x.shape, data)

### constrain(loss) section



Defines a set of loss functions that are used to train a machine learning model for solving partial differential equations (PDEs). The loss functions are defined as Python functions that take in the predicted solution `u` and the input data `x` and `t`, which represent the spatial and temporal coordinates of the PDE.

The first function, `l2_loss`, computes the mean squared error between the predicted solution and the true solution. This is a common loss function used in regression problems.

The next three functions, `bcloss`, `icloss`, and `eqloss`, compute the loss for the boundary conditions, initial conditions, and interior equations, respectively. The `bcloss` function computes the difference between the predicted solution and a constant value of 0 at the boundary points. The `icloss` function computes the difference between the predicted solution and a sine wave at the initial time. The `eqloss` function computes the difference between the predicted solution and the right-hand side of the PDE.

The final function, `loss`, combines the individual loss functions into a total loss using a weighted sum. The weights for each type of input are specified in the `loss_weights` dictionary. The `loss` function takes in a dictionary of inputs, which includes the boundary conditions, initial conditions, and interior equations. The `loss` function computes the loss for each type of input and combines them into a total loss using a weighted sum. The total loss is returned along with a dictionary of individual losses for each type of input.

Overall, these loss functions are used to train a machine learning model to solve a given PDE. The model is trained to predict the solution to the PDE at a set of time and space coordinates. The loss function is used to optimize the model parameters to minimize the difference between the predicted solution and the true solution.

In [ ]:
import torch

def l2_norm(x):
    return torch.mean(x ** 2)

def bcloss(u, x, t):
    return l2_norm(u - 0)

def icloss(u, x, t):
    return l2_norm(u - torch.sin(torch.pi * x))

nu = 0.01 / torch.pi

def eqloss(u, x, t):
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    return l2_norm(u_t + u * u_x - nu * u_xx)


loss_weights = {"boundary": 1.0, "initial": 1.0, "interior": 1.0}
def loss(fn, data):
    u = {k: fn(**v) for k, v in data.items()}
    loss_dict = {
        "boundary": bcloss(u["boundary"], **data["boundary"]),
        "initial": icloss(u["initial"], **data["initial"]),
        "interior": eqloss(u["interior"], **data["interior"]),
    }
    total_loss = sum([loss_dict[k] * loss_weights[k] for k in loss_dict.keys()])

    return total_loss, loss_dict

### model setup

Define a neural network model for solving partial differential equations (PDEs) using PyTorch. The model is a multi-layer perceptron (MLP) that takes in two inputs, `t` and `x`, which represent the temporal and spatial coordinates of the PDE.

In [ ]:
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(2, 128),
    nn.Tanh(),
    nn.Linear(128, 128),
    nn.Tanh(),
    nn.Linear(128, 1),
)

def u_sol(t, x):
    u = model(torch.cat([t[:, None], x[:, None]], dim=-1))
    return u.squeeze()

The model is defined using the `nn.Sequential` class, which allows us to define a sequence of layers that are applied to the input data in order.

The MLP consists of three fully connected layers, each followed by a hyperbolic tangent activation function. The first layer takes in two inputs, `t` and `x`, and outputs 32 hidden units. The second layer also has 32 hidden units, and the final layer outputs a single value, which represents the predicted solution to the PDE at the given time and space coordinates.

The `u_sol` function takes in two arguments, `t` and `x`, which represent the time and space coordinates of the PDE. The function uses the trained model to predict the solution to the PDE at the given coordinates. The `torch.cat()` function is used to concatenate the time and space coordinates along the last dimension, and the `squeeze()` method is used to remove any extra dimensions from the output.

Overall, this code defines a neural network model for solving PDEs and provides a function for using the trained model to predict the solution to a given PDE at a set of time and space coordinates. The MLP architecture can be modified to suit different types of PDEs, and the `u_sol` function can be used to evaluate the performance of the model on a given PDE.

### Start training

First define optimizer:

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.01
)

Start Training!

In [ ]:
def set_requires_grad(tensor_dict, requires_grad=True, device="cpu"):
    """
    Set the requires_grad attribute of all tensors in a nested dictionary.

    Args:
        tensor_dict (dict): A nested dictionary containing tensors.
        requires_grad (bool): Whether to require gradient computation or not.
        device (str): The device to move tensors to.
    """
    _tensor_dict = {}
    for k, v in tensor_dict.items():
        if isinstance(v, dict):
            _tensor_dict[k] = set_requires_grad(v, requires_grad, device=device)
        else:
            # _tensor_dict[k] = v.clone().detach().requires_grad_(True).to(device)
            _tensor_dict[k] = torch.tensor(
                v,dtype=torch.float, device=device, requires_grad=requires_grad
            )
    del tensor_dict
    return _tensor_dict

In [ ]:
model = model.train()
for i in range(100):
    inputs = set_requires_grad(get_data())

    optimizer.zero_grad()
    _loss, loss_dict = loss(u_sol, inputs)
    _loss.backward()
    optimizer.step()


    _loss_dict = {k: v.item() for k, v in loss_dict.items()}
    print(f"Step {i}, total loss {_loss.item():.4f}, loss_dict {_loss_dict}")

The code above shows a training loop for a machine learning model that solves partial differential equations (PDEs). The loop iterates over a fixed number of steps (in this case, 100), and updates the model parameters at each step using the Adam optimizer.

At each step of the loop, a new set of input data is generated using the `get_data()` function. The `set_requires_grad()` function is then called to set the `t` and `x` inputs to require gradients, which allows the optimizer to update their values during training.

The optimizer's `zero_grad()` method is called to reset the gradients of all model parameters to zero. The `loss()` function is then called to compute the loss function for the current set of input data. The loss function returns both the total loss and a dictionary of individual losses for each type of input data (boundary conditions, initial conditions, and interior equations).

The `backward()` method is called on the total loss to compute the gradients of all model parameters with respect to the loss. The optimizer's `step()` method is then called to update the model parameters based on the gradients and the specified hyperparameters.

Finally, the total loss and individual losses are printed to the console for monitoring the training process. The total loss is printed as a floating-point number, and the individual losses are printed as a dictionary of floating-point numbers.

Overall, this code shows a basic training loop for a machine learning model that solves PDEs. The loop can be modified to suit different types of PDEs and input data, and the optimizer's hyperparameters can be adjusted to improve the performance of the model.

## 4. Test

In [ ]:
x = torch.linspace(-1, 1, 500)
t = torch.linspace(0, 1, 11)

Next cell is used to evaluate the trained neural network model on a given set of time and space coordinates. Specifically, the code computes the predicted solution to a PDE at a single time point, `t[t_idx]`, and a set of spatial coordinates, `x`.

The `u_sol` function is called with two arguments: a tensor of ones with the same shape as `x` multiplied by `t[t_idx]`, and `x` itself. This creates a tensor of shape `(n_points,)`, where `n_points` is the number of spatial coordinates in `x`. The `u_sol` function uses the trained neural network model to predict the solution to the PDE at each of these spatial coordinates and the given time point.

The predicted solution is returned as a tensor of shape `(n_points,)`, which represents the predicted values of the solution at each of the spatial coordinates. This tensor is assigned to the variable `u`.

Overall, this code is a simple example of how to use the trained neural network model to predict the solution to a PDE at a given set of time and space coordinates. The `u_sol` function can be called with different time and space coordinates to evaluate the model on different parts of the domain.

In [ ]:
t_idx = 0

u = u_sol(torch.ones_like(x)*t[t_idx], x)

Write code to plot output:

In [ ]:
plt.plot(x, u.detach().numpy())